# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

* [1. Откройте и изучите файл](#section_id1)</a>
* [2. Разбейте данные на выборки](#section_id2)</a>
* [3. Исследуйте модели](#section_id3)</a>
* [4. Проверьте модель на тестовой выборке](#section_id4)</a>
* [5. Проверьте модели на адекватность](#section_id5)</a>
* [Вывод](#section_id7)</a>
* [6. Чек-лист готовности проекта](#section_id6)</a>

## Откройте и изучите файл
<a id='section_id1'></a>

In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


df
Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Резюме.

Мы открыли и изучили код. Предобработку даннфых делатьнебудем поскольку в задании сказано, что этого ненадо.

## Разбейте данные на выборки
<a id='section_id2'></a>

In [6]:
# Сначала разделим на валидационную и обучающую выборку.
# отделим 40% данных для валидационной выборки
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
# извлекаем признаки 
features = df_valid.drop('is_ultra', axis=1) # извлекаем признаки 
target = df_valid['is_ultra']# извлекаем целевой признак

In [7]:
# Теперь отделим 50% данных от валидационной выборки
features_test, features_valid, target_test, target_valid = train_test_split(
    features, target, test_size=0.5, random_state=12345)
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

Резюме.

Разбили данные сначала на две части в 40% и 60%. После 40% еще пополам.

In [8]:
print(df_train.shape[0]) # количество строк
print(df_train.shape[1]) # количество столбцов

1928
5


In [9]:
df_valid.shape # первое число количество строк, второе количество столбцов

(1286, 5)

## Исследуйте модели
<a id='section_id3'></a>

Логистическая регрессия

In [10]:
# Обучите модель вызовом метода fit().
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
model.fit(features_train, target_train)

# Чтобы посмотреть accuracy модели, нужно вызвать функцию .score():
accuracy = model.score(features_valid, target_valid)
print("Accuracy лучшей моделий логистической регрессии:", accuracy)
# Алгоритм 'lbfgs' — один из самых распространённых. Он подходит для большинства задач.
# Гиперпараметром max_iter задаётся максимальное количество итераций обучения. Значение этого параметра по умолчанию равно 100.

Accuracy лучшей моделий логистической регрессии: 0.6842923794712286


Модель решающего дерева

In [11]:
est_tree_model = None
best_tree_result = 0
for depth in range(1, 7):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучили модель с заданной глубиной дерева
    model_tree.fit(features_train, target_train) # обучили модель

    result_tree = model_tree.score(features_valid, target_valid) # посчитали качество модели
    if result_tree > best_tree_result:
        best_tree_model = model_tree
        best_tree_result = result_tree
        
print("Accuracy лучшей модели решающего дерева:", best_tree_result)

Accuracy лучшей модели решающего дерева: 0.7838258164852255


Случайный лес

In [12]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=est, min_samples_leaf=4) # обучите модель с заданным количеством деревьев
    model.fit(features_train,target_train) # обучите модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитали качество модели на валидационной выборке
    if result > best_result:
        best_model = model# сохранили наилучшую модель
        best_result = result#  сохранили наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели на валидационных данных:", best_result)   

Accuracy наилучшей модели на валидационных данных: 0.8055987558320373


Резюме.

Наилучшей модель с максимально большим значением accuracy оказался случайный лес со значением 0.805. Модель решающего дерева 0.783, а логистическая регрессия показала самый низкий результат в 0.684.

## Проверьте модель на тестовой выборке
<a id='section_id4'></a>

Логистическая регрессия

In [13]:
# Обучите модель вызовом метода fit().
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
model.fit(features_train, target_train)

# Чтобы посмотреть accuracy модели, нужно вызвать функцию .score():
accuracy = model.score(features_test, target_test)
print("Accuracy лучшей моделий логистической регрессии:", accuracy)

Accuracy лучшей моделий логистической регрессии: 0.7107309486780715


Решающее дерево

In [14]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=8) # обучите модель с заданной глубиной дерева
model_tree.fit(features_train, target_train) # обучите модель
result_tree = model_tree.score(features_test, target_test) # посчитали качество модели на тестовой выборке
print("Accuracy лучшей модели решающего дерева:", result_tree)  

Accuracy лучшей модели решающего дерева: 0.7791601866251944


Случайный лес

In [15]:
model = RandomForestClassifier(max_depth=8, random_state=12345, n_estimators=40, min_samples_leaf=1) # обучите модель с заданным количеством деревьев
model.fit(features_train,target_train) # обучите модель на тренировочной выборке
result = model.score(features_test, target_test) # посчитали качество модели на тестовой выборке
print("Accuracy наилучшей модели на тестовых данных:", result) 

Accuracy наилучшей модели на тестовых данных: 0.8087091757387247


Резюме.

Были проверены модели на тестовой выборки. Результаты получились похожие

# Вывод
<a id='section_id7'></a>

Были открыты и изучен файл. После разбили данные на выборки. Вначале на две в 40% и 60%. После 40% еще раз разбили попалам. Были проверены три модели: логистическая регрессия, решающее дерево, случайный лес. Лучший результат показал случайный лес (0.79), худшуу логическая регресия (0.68). Была проверена модель на тестовой выборке. Наилучшее качество (0.8087) достигается при использовании случайного леса с гиперпараметрами max_depth=7, n_estimators=40, min_samples_leaf=1.